## OUTLIERS

In [1]:
import random
import scipy.stats as st
import numpy as np
import math
from math import gamma, pi
import time
import scipy
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import numpy.random as rnd
import pickle
import os.path
from deap import creator, base, tools, algorithms
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import scipy.linalg as la

In [2]:
def getAllSpansEqual(numOfDims, spanForAll):
    return np.ones(numOfDims)*spanForAll

def getRandomUniformSpans(numOfDims, low, high):
    return np.random.rand(numOfDims)*(high-low) + low

def getVolumeElipsoid(params):
    nDims = params.size
    return pow(pi, (nDims/2)) / gamma(nDims/2 + 1) * np.prod(params)

def getVolumeElipsoid2(params):
    nDims = params.size
    return (2/nDims)*pow(pi, (nDims/2)) / gamma(nDims/2) * np.prod(params)

# print(getVolumeElipsoid(np.array( [1,5,4])))
# print(getVolumeElipsoid2(np.array( [1,5,4])))

def fitEllipsoid_dumb(points, elipsoidParameters):
    d = np.size(elipsoidParameters)
    elipsoidParameters.shape=(1,d)
    
    max_dist = np.max( np.sum(np.square(np.divide(points,elipsoidParameters)),1) )
    increase_factor = math.sqrt(max_dist) + 0.00001
    print("increase_factor", increase_factor)
    new_ellipsoid_parameters = increase_factor*elipsoidParameters
    return new_ellipsoid_parameters

def isInElipsoid(points, elipsoidParameters):
    # points is a d x num_p numpy matrix where d is the number of dimensions and num_p is the number of points.
    # elipsoidParameters is an d-dimensional array, where each element is the RADIUS of the axis.
    d = np.size(elipsoidParameters)
    elipsoidParameters.shape=(1,d) 
    return np.sum(np.square(np.divide(points,elipsoidParameters)),1) <= 1

In [3]:
def minVolEllipse(P, tolerance ):
    # P = d x numP ( points )
    # tolerance = skalar ( priporočeno = 0.0004 )

    d, N = np.shape(P)
    Q = np.ones((d+1, N))
    Q[0:d,:] = P

    # initializations
    # -----------------------------------
    count = 1
    err = 1
    u = (1/N) * np.ones((N,1))          # 1st iteration
    
    while (err > tolerance):
        X = np.dot( np.dot( Q   , np.diagflat(u))    , Q.T ) # % X = \sum_i ( u_i * q_i * q_i')  is a (d+1)x(d+1) matrix
        M = (np.dot( Q.T , np.linalg.inv(X) ) * Q.T).sum(-1)
        # print("M", M)
        M.shape=(N,1)
        j = np.argmax(M)
        maximum = M[j,0]
        step_size = (maximum - d -1)/((d+1)*(maximum-1))
        new_u = (1 - step_size)*u
        new_u[j] = new_u[j] + step_size
        count += 1
        err = np.linalg.norm(new_u - u)
        u = new_u
        print("err", err)
        
    U = np.diagflat(u)
    print("done, err final =", err, ", iterations needed:", count )

    # the A matrix for the ellipse
    # --------------------------------------------
    Pu = np.dot(P,u)
    C = (1/d) * np.linalg.pinv( np.dot( np.dot( P , U) , P.T ) - np.dot(Pu, Pu.T ))

    # center of the ellipse 
    # --------------------------------------------
    b = np.dot(P , u)
    return C, b

In [4]:
# cube.
# 1,2   -> 3

def getOutliersCube(train_1, train_2, test, bits0123 ):
    model_index = bits0123     
    model_str = '0'+str(model_index+1)+'_'
    
    region_train = []
    region_test = []
    

    base_path_train = os.path.join("..", train_1 )
    for i in range(0, 10):
        region_train.append(os.path.join( base_path_train, model_str+"bioproc_Region0ViableSet_Iter" + str(i+1) + ".p"))
        
    base_path_train = os.path.join("..", train_2 )
    for i in range(0, 10):
        region_train.append(os.path.join( base_path_train, model_str+"bioproc_Region0ViableSet_Iter" + str(i+1) + ".p"))
        
    base_path_test = os.path.join("..", test )
    for i in range(0, 10):
        region_test.append(os.path.join( base_path_test, model_str+"bioproc_Region0ViableSet_Iter" + str(i+1) + ".p"))

    print(region_train)
    print()
    print(region_test)

    viableTrainPoints = []  
    for region_file in region_train: 
        viablePointsRegion = pickle.load(open(region_file, "rb")) 
        viableTrainPoints.extend(viablePointsRegion)
    viableTrainPoints = np.array(viableTrainPoints)
    
    
    
    
    viableTestPoints = []  
    for region_file in region_test: 
        viablePointsRegion = pickle.load(open(region_file, "rb")) 
        viableTestPoints.extend(viablePointsRegion)
    viableTestPoints = np.array(viableTestPoints)
    
    nDims = viableTrainPoints.shape[1]
    # print("nDims", nDims)

    pca = PCA(n_components=nDims)
    pca.fit( viableTrainPoints )
    transformedTrainViable = pca.transform(viableTrainPoints)
    transformedTestViable = pca.transform(viableTestPoints)

    minP = np.min(transformedTrainViable, axis=0)
    maxP = np.max(transformedTrainViable, axis=0)
    dP = maxP - minP
    volCube = np.prod(dP) 
    
    print(transformedTrainViable.shape)
    
    num_ok = 0
    not_ok = 0
    for i in range( transformedTestViable.shape[0] ):
        point = transformedTestViable[i,:]
        isGood = np.all(point >= minP ) & np.all(point <= maxP )
        if isGood:
            num_ok += 1
        else:
            not_ok += 1
    
    sumi = num_ok + not_ok
    
    frac = not_ok / sumi
    
    print("volCube                        :", volCube )
    print("number of all training examples:", viableTrainPoints.shape[0] )
    print("number of all tested   examples:", viableTestPoints.shape[0] )
    print("number of inliers              :", num_ok )
    print("number of outliers             :", not_ok )
    print("fraction of outliers           :", frac )
    print("checksum                       :", sumi )
    
    return num_ok, not_ok


In [5]:
# "results_opt", "results_opt_rep1", "results_opt_rep2"

train_1 = "results_opt"
train_2 = "results_opt_rep1"
test = "results_opt_rep2"
bits0123 = 0

t1,f1 = getOutliersCube(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt\\01_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter7.p', '..\\result

In [6]:
# "results_opt", "results_opt_rep1", "results_opt_rep2"

train_1 = "results_opt_rep1"
train_2 = "results_opt_rep2"
test = "results_opt"
bits0123 = 0

t2,f2 = getOutliersCube(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep2\

In [7]:
# "results_opt", "results_opt_rep1", "results_opt_rep2"

train_1 = "results_opt"
train_2 = "results_opt_rep2"
test = "results_opt_rep1"
bits0123 = 0

t3, f3 = getOutliersCube(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt\\01_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter7.p', '..\\result

In [8]:
# cube.
# 1,2   -> 3

def getOutliersEllipsoid(train_1, train_2, test, bits0123 ):
    model_index = bits0123     
    model_str = '0'+str(model_index+1)+'_'
    
    region_train = []
    region_test = []
    

    base_path_train = os.path.join("..", train_1 )
    for i in range(0, 10):
        region_train.append(os.path.join( base_path_train, model_str+"bioproc_Region0ViableSet_Iter" + str(i+1) + ".p"))
        
    base_path_train = os.path.join("..", train_2 )
    for i in range(0, 10):
        region_train.append(os.path.join( base_path_train, model_str+"bioproc_Region0ViableSet_Iter" + str(i+1) + ".p"))
        
    base_path_test = os.path.join("..", test )
    for i in range(0, 10):
        region_test.append(os.path.join( base_path_test, model_str+"bioproc_Region0ViableSet_Iter" + str(i+1) + ".p"))

    print(region_train)
    print()
    print(region_test)

    viableTrainPoints = []  
    for region_file in region_train: 
        viablePointsRegion = pickle.load(open(region_file, "rb")) 
        viableTrainPoints.extend(viablePointsRegion)
    viableTrainPoints = np.array(viableTrainPoints).T
    
    viableTestPoints = []  
    for region_file in region_test: 
        viablePointsRegion = pickle.load(open(region_file, "rb")) 
        viableTestPoints.extend(viablePointsRegion)
    viableTestPoints = np.array(viableTestPoints).T
    
    nDims = viableTrainPoints.shape[0]
    print("nDims", nDims)
    # print(viableTrainPoints.shape)
    # print(viableTestPoints.shape)
    
    P = viableTrainPoints
    tolerance = 0.001

    C, b = minVolEllipse( P, tolerance )

    C_minus = C/(1-np.dot( b.T, np.dot(C,b) )  )
    (eigvals,eigvecs) = la.eig(C_minus)
    
    P2_train = np.dot( eigvecs.T, (P - b))
    P2_test = np.dot( eigvecs.T,  ( viableTestPoints  - b))

    R = 1./np.sqrt(np.abs(eigvals))
    R.shape = (np.size(R), 1)

    R2 = R*np.sqrt( np.max(np.sum(np.square(np.divide( P2_train, R )),0)) )
    score_good = np.max(np.sum(np.square(np.divide( P2_train, R2 )),0))
    
    vol_good = getVolumeElipsoid(R2)
    
    P2_test_scores = np.sum(np.square(np.divide( P2_test, R2 )),0)
    num_ok = np.sum( (P2_test_scores <= 1)*1 )
    not_ok = np.sum( (P2_test_scores > 1)*1 )
    
    sumi = num_ok + not_ok
    
    frac = not_ok / sumi
    
    print("score_Ellipse_train              :", score_good)
    print("volEllipse                       :", vol_good )
    print("number of all training examples  :", viableTrainPoints.shape[1] )
    print("number of all tested   examples  :", viableTestPoints.shape[1] )
    print("number of inliers                :", num_ok )
    print("number of outliers               :", not_ok )
    print("fraction of outliers             :", frac )
    print("checksum                         :", sumi )
    
    return num_ok, not_ok


In [11]:
# "results_opt", "results_opt_rep1", "results_opt_rep2"

train_1 = "results_opt"
train_2 = "results_opt_rep1"
test = "results_opt_rep2"
bits0123 = 0

e_t1,e_f1 = getOutliersEllipsoid(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt\\01_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter7.p', '..\\result

err 0.003227772537144737
err 0.003555117244097469
err 0.0035737223994937446
err 0.0037309798723519066
err 0.003821967043361359
err 0.0033996754610674533
err 0.003511003453452733
err 0.0035013224688616846
err 0.003305627105871116
err 0.0031380944868135098
err 0.003333036042907988
err 0.0030471475291642134
err 0.003021336359444599
err 0.0031732820358813846
err 0.0032546118842842944
err 0.0032963864643697598
err 0.0033946985605351468
err 0.003283706980076405
err 0.0028351401291921846
err 0.0030002230971608685
err 0.0032105270599548214
err 0.003410971275121555
err 0.0033104446493901047
err 0.003223217119988472
err 0.0033436441359027333
err 0.0033328874474602903
err 0.0032139139783089153
err 0.0032512760617848524
err 0.0031272798297300453
err 0.002902676529639028
err 0.00268105146845626
err 0.002833229944576866
err 0.0029354997555054
err 0.002948485451299303
err 0.0030670123921861647
err 0.003425484819118599
err 0.003510291958858483
err 0.00328149564587305
err 0.003339702350000883
err 0.003

err 0.001508936377003162
err 0.001654020836890881
err 0.001748145019773018
err 0.0016580330945790164
err 0.0017261378142435031
err 0.0018062455955114686
err 0.0017625173169251134
err 0.001737892146711308
err 0.001686115162880141
err 0.0017243309284449007
err 0.0017710380945556378
err 0.0016224018621254453
err 0.0018079459287058009
err 0.0014713348065121825
err 0.0015610646936119587
err 0.0015923537129080107
err 0.0016376597029687777
err 0.0017274352607167874
err 0.0017389792788083727
err 0.0015308035085900861
err 0.0014553129897229215
err 0.0014176351410590436
err 0.001483966809893445
err 0.0015119701193641337
err 0.0014781019063487273
err 0.001521875165648875
err 0.0016162189457278983
err 0.001546726196836132
err 0.0015923858178231898
err 0.0015539023931672915
err 0.001626185032757571
err 0.0016560924051860413
err 0.0017621132100639188
err 0.0015382256970594716
err 0.001576297559947432
err 0.0015668519025920104
err 0.0015837408866596235
err 0.001436873939069004
err 0.00154512401975978

In [12]:
# "results_opt", "results_opt_rep1", "results_opt_rep2"

train_1 = "results_opt"
train_2 = "results_opt_rep2"
test = "results_opt_rep1"
bits0123 = 0

e_t2,e_f2= getOutliersEllipsoid(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt\\01_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\01_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter7.p', '..\\result

err 0.004004323485489072
err 0.003994591377853059
err 0.004198812052636853
err 0.003514713035136584
err 0.0035270663253862706
err 0.0033547556220642314
err 0.0033774447034598505
err 0.0036147647110887585
err 0.003389729443260402
err 0.0034586282025027663
err 0.00352008872037025
err 0.003596655376783682
err 0.003126537083196505
err 0.0030368065675965812
err 0.0031286961006958157
err 0.0033183514288489357
err 0.003313046570597239
err 0.0034906211386483673
err 0.003871409657887105
err 0.0035503540921084306
err 0.003856834321273978
err 0.003601493437149166
err 0.003297608345897132
err 0.003369965563596804
err 0.0031297762969919847
err 0.0032512243792712763
err 0.0031903178065855364
err 0.0031259373838063388
err 0.0029740406814161256
err 0.0031302608304350724
err 0.002907892290836059
err 0.002997752472994659
err 0.0031808407573735357
err 0.003338395194397782
err 0.003248594556952255
err 0.0030605355194075826
err 0.003153282322488117
err 0.0033482730124380156
err 0.0029258148966805697
err 0.

err 0.0017771997858684742
err 0.0017510844255861269
err 0.0017067927102211366
err 0.0016833875613109896
err 0.0016261817754582206
err 0.0014517372459682484
err 0.0015773334979158288
err 0.0015286426863432927
err 0.0016243309580974998
err 0.0015676985273208443
err 0.0015103415562984482
err 0.0015823542722366321
err 0.0016915747088116248
err 0.0015405252476796
err 0.0015504382926890305
err 0.0016386868061105445
err 0.001681129185802254
err 0.001526519979813106
err 0.0015313113707088638
err 0.0015402153444495587
err 0.0015928002818176089
err 0.0015267073483145259
err 0.0016227619785779837
err 0.0016843061926185598
err 0.001507039939805132
err 0.001560086231508621
err 0.0016956420342231974
err 0.0016926758304670567
err 0.0015636720739914515
err 0.0015644878246013052
err 0.0015565122340193774
err 0.0016057075705469006
err 0.0015906146276954357
err 0.0015746383388403056
err 0.0013980092552580742
err 0.0014952275358915451
err 0.00145353795590721
err 0.001378065254234117
err 0.0014567933606219

In [13]:
# "results_opt", "results_opt_rep1", "results_opt_rep2"

train_1 = "results_opt_rep1"
train_2 = "results_opt_rep2"
test = "results_opt"
bits0123 = 0

e_t3,e_f3 = getOutliersEllipsoid(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt_rep1\\01_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep2\\01_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep2\

err 0.0036696651596967353
err 0.003837017860586611
err 0.004015731547848506
err 0.003985932932372025
err 0.0032302476434211257
err 0.003451262910089096
err 0.0033909468359455647
err 0.0034169155161494157
err 0.0035101645380690386
err 0.003586946481555638
err 0.0034461922157920334
err 0.0034618141994874826
err 0.0034893771335457895
err 0.0035891158208457476
err 0.003242195272129317
err 0.0033152359309567125
err 0.00353768013099724
err 0.003533648859389554
err 0.003406189599744454
err 0.0034713863545873204
err 0.0036210545893908463
err 0.0033968802258845577
err 0.00326316378369311
err 0.0034307554465204796
err 0.0032737510453574336
err 0.0032469171678982975
err 0.0029821715920833694
err 0.003013802855356407
err 0.003119606853132576
err 0.003277344720877971
err 0.0034625064489438883
err 0.003002042053246495
err 0.0030442902144913724
err 0.003190899729881519
err 0.003228367440978112
err 0.003016554550435928
err 0.0029997515008647183
err 0.0031557972034135053
err 0.003201893535275704
err 0.

err 0.0014675857470687775
err 0.0015022045797131785
err 0.0014430427273176255
err 0.0014837872603886762
err 0.0015077156408722417
err 0.001445389800615545
err 0.0015699427640927736
err 0.0016668066848120897
err 0.0015358219722128013
err 0.0016572899586146824
err 0.001521752049027023
err 0.001636654072031228
err 0.00167702064057247
err 0.0017007403459366746
err 0.0017513185285156975
err 0.0017462064663337784
err 0.0015500071770160504
err 0.0015738354603855953
err 0.001694425160407261
err 0.0014767401230961043
err 0.0015129299912975773
err 0.001581544938757015
err 0.0016464113975375226
err 0.001626458410344563
err 0.0014772368129095795
err 0.001502288192011007
err 0.0014510255270984466
err 0.0014516041924971786
err 0.001523537571829023
err 0.001601507767960175
err 0.0013495423721759591
err 0.00147814595606992
err 0.001488243255226895
err 0.001510141813885662
err 0.0014258691490396358
err 0.0015150737093368991
err 0.0014851774630217024
err 0.0015134799111199005
err 0.0016311933414619086
e

In [48]:
cube_score_bit1 = ( t1 + t2 + t3)/( (f1 + f2 + f3) + ( t1 + t2 + t3) )
print("cube_score_bit1     :", cube_score_bit1)
ellipsoid_score_bit1 = ( e_t1 + e_t2 + e_t3)/( (e_f1 + e_f2 + e_f3) + ( e_t1 + e_t2 + e_t3) )
print("ellipsoid_score_bit1:", ellipsoid_score_bit1)

cube_score_bit1     : 0.97949258328605
ellipsoid_score_bit1: 0.9613358190808138


## 2-bitni procesor: inliers

In [21]:
train_1 = "results_opt_rep1"
train_2 = "results_opt_rep2"
test = "results_opt"
bits0123 = 1

t1_bit2,f1_bit2 = getOutliersCube(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep2\

In [22]:
train_1 = "results_opt"
train_2 = "results_opt_rep2"
test = "results_opt_rep1"
bits0123 = 1

t2_bit2,f2_bit2 = getOutliersCube(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt\\02_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter7.p', '..\\result

In [23]:
train_1 = "results_opt"
train_2 = "results_opt_rep1"
test = "results_opt_rep2"
bits0123 = 1

t3_bit2,f3_bit2 = getOutliersCube(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt\\02_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter7.p', '..\\result

In [24]:
train_1 = "results_opt_rep1"
train_2 = "results_opt_rep2"
test = "results_opt"
bits0123 = 1

e_t1_bit2,e_f1_bit2 = getOutliersEllipsoid(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep2\

err 0.0036112599451533643
err 0.0037068124632649484
err 0.0034674092204629893
err 0.0036038172237907715
err 0.003795423325581269
err 0.003607049112812599
err 0.003551009603645766
err 0.0031002241333645906
err 0.003301493661337565
err 0.0035213038735869076
err 0.003663991791051974
err 0.0034166949672725876
err 0.0035562190826014207
err 0.0035489654767094114
err 0.003702355702538616
err 0.0033773914822974
err 0.003302724004620575
err 0.0030407646374258275
err 0.002945708865797431
err 0.003038011644670546
err 0.003139147764397236
err 0.003337856739264468
err 0.00320000024277927
err 0.0033062407487606414
err 0.0034815321469604315
err 0.0034740281036498674
err 0.003733885590311641
err 0.0034805113580684815
err 0.003398937294328899
err 0.003281731130726134
err 0.003358813206632823
err 0.003260066153424328
err 0.003108624030635911
err 0.00339046035246532
err 0.0033667221222155675
err 0.0035322778250410237
err 0.003545252689403063
err 0.0029765160546928915
err 0.0030732932765753615
err 0.00304

err 0.0017159326017480994
err 0.0017848669038165086
err 0.0017944511327311133
err 0.0017843919251677698
err 0.0018002823968461647
err 0.0017476389199861974
err 0.0016121569618994653
err 0.0016056030280473766
err 0.0015296373376373323
err 0.0014893931888319506
err 0.0016206074156873533
err 0.0015683883557668654
err 0.0016923589209801146
err 0.0015549575783218791
err 0.0015738637154037861
err 0.001583068751992934
err 0.001630743204799912
err 0.0015877509856659254
err 0.0016091457764294372
err 0.001639137785815105
err 0.0015292914381413727
err 0.0015269779660249448
err 0.0014817799003140088
err 0.0015496636768067824
err 0.0014692736900758733
err 0.0015993900486126588
err 0.0015336094269296629
err 0.0016052516684160298
err 0.001528815149828577
err 0.0015084562521255288
err 0.0015763162358042864
err 0.0017093705821029908
err 0.001550339569479158
err 0.00162595649964894
err 0.0015131720739444978
err 0.0016808616698110513
err 0.0016916991867387098
err 0.0017631470765033731
err 0.0015665892246

In [25]:
train_1 = "results_opt"
train_2 = "results_opt_rep2"
test = "results_opt_rep1"
bits0123 = 1

e_t2_bit2,e_f2_bit2 = getOutliersEllipsoid(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt\\02_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep2\\02_bioproc_Region0ViableSet_Iter7.p', '..\\result

err 0.003982986436031678
err 0.003948991955933389
err 0.0037983684402995545
err 0.0038402597489267634
err 0.0036578226470769175
err 0.003293795564021798
err 0.0033543092647047076
err 0.003403415308333921
err 0.0034768905120001778
err 0.0034704648786397676
err 0.0033100214514023836
err 0.003413223164906931
err 0.003474096844563766
err 0.0034271951672286086
err 0.0033962917716078827
err 0.0034265816834493385
err 0.0034521368362262776
err 0.003501762842005434
err 0.003674367245348578
err 0.0033800690756817828
err 0.0036306407681675795
err 0.0034868272226617576
err 0.0031134386466505205
err 0.0030750190534879224
err 0.003219158364627616
err 0.0030627346694630016
err 0.003317109855965147
err 0.003421008102934949
err 0.003532833054659981
err 0.003695794605255464
err 0.003574416983407063
err 0.003752232365113817
err 0.003812317556633793
err 0.0033280039701113744
err 0.003085255877157469
err 0.003115475778680618
err 0.0029544392816981714
err 0.0029377591922677172
err 0.0030657370051351005
err 

err 0.0016697096908388784
err 0.0017371255715265012
err 0.001589250887381577
err 0.0016741372883469557
err 0.0016075975772513477
err 0.001444213667243353
err 0.0014782726874537587
err 0.001596263626472608
err 0.0017602540186079344
err 0.0016803724035727666
err 0.0017279998219473717
err 0.0015439067131225483
err 0.0015117983348215801
err 0.0014568237488854646
err 0.001550378809575994
err 0.0014525183364393216
err 0.001433797441931077
err 0.0013792669731578495
err 0.001284140469613617
err 0.0013724669763167314
err 0.0014308471305567903
err 0.0014500308255619339
err 0.0015156238578706352
err 0.001441366241799318
err 0.0015196933055018256
err 0.0015964974340407902
err 0.0016250166003230905
err 0.0017595292473091208
err 0.001732557867949337
err 0.00176332941707242
err 0.001633656432851019
err 0.0016327525825473334
err 0.001598143335262039
err 0.0015011479826146231
err 0.0015087123031734479
err 0.0015028742459862324
err 0.001384595570289082
err 0.001486368359890409
err 0.001522836448801673
e

In [29]:
train_1 = "results_opt"
train_2 = "results_opt_rep1"
test = "results_opt_rep2"
bits0123 = 1

e_t3_bit2,e_f3_bit2 = getOutliersEllipsoid(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt\\02_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\02_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\02_bioproc_Region0ViableSet_Iter7.p', '..\\result

err 0.0038041898539151664
err 0.003782247849975527
err 0.003379242296919309
err 0.0034977161599757723
err 0.0033778935805844823
err 0.0030884401445981785
err 0.003047392262527707
err 0.0031636936595696853
err 0.0029302599123939466
err 0.003055966323478797
err 0.0031394422642793064
err 0.0033216748288457066
err 0.003415911665014355
err 0.003425447014630348
err 0.003497475611356206
err 0.0035182696985153
err 0.003627064587269946
err 0.003703056604208935
err 0.003583543582345056
err 0.0033954713138333917
err 0.0035457493863602437
err 0.0036599156395837875
err 0.0033705273100003252
err 0.003563750310189167
err 0.003442128479608247
err 0.0032102239279334227
err 0.0032639028668798174
err 0.0030175117810280778
err 0.0030740672812179437
err 0.003110934606478991
err 0.0031267734767726077
err 0.0031985362348793773
err 0.0031834464054858046
err 0.0031597595489713686
err 0.002966427169336736
err 0.002908461467667103
err 0.0030375755193259367
err 0.0028979880226287108
err 0.002793291251675542
err 0

err 0.0017918438441955026
err 0.0017599182206437516
err 0.0015072832743436362
err 0.001582715957635076
err 0.0014454628052329949
err 0.0015275714331363034
err 0.0015696682698260415
err 0.00162375729549048
err 0.0014782175424003295
err 0.0014384964623152753
err 0.001524576481179439
err 0.0015987412786930605
err 0.0016463846572721853
err 0.0016768326073866647
err 0.0016584566590465723
err 0.0016751396446472268
err 0.0017215980596023666
err 0.001769733667072602
err 0.0015968014368743734
err 0.0015990225817614106
err 0.0017003922725736644
err 0.0017225870097343236
err 0.0015107720550425802
err 0.0015686227631224552
err 0.0015079955594563643
err 0.001482850454588334
err 0.0014033046927456223
err 0.0014479985565982144
err 0.0015037885506520069
err 0.0014843300595673615
err 0.0014737812013122529
err 0.0015037152195617731
err 0.001470575356890399
err 0.0013968647633988056
err 0.0015030417781779963
err 0.0015346149692269334
err 0.0015679275046784525
err 0.0014696847169434503
err 0.0015686416863

In [54]:
cube_score_bit2 = ( t1_bit2 + t2_bit2 + t3_bit2)/( (f1_bit2 + f2_bit2 + f3_bit2) + ( t1_bit2 + t2_bit2 + t3_bit2) )
print("cube_score_bit2     :", cube_score_bit2 )
ellipsoid_score_bit2 = ( e_t1_bit2 + e_t2_bit2 + e_t3_bit2)/((e_f1_bit2+e_f2_bit2+e_f3_bit2)+(e_t1_bit2+e_t2_bit2+e_t3_bit2))
print("ellipsoid_score_bit2:", ellipsoid_score_bit2 )

cube_score_bit2     : 0.9937022645048907
ellipsoid_score_bit2: 0.9858635937290634


## 3-bitni procesor: inliers

In [26]:
train_1 = "results_opt_rep1"
train_2 = "results_opt_rep2"
test = "results_opt"
bits0123 = 2

t1_bit3, f1_bit3 = getOutliersCube(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep2\

In [27]:
train_1 = "results_opt"
train_2 = "results_opt_rep2"
test = "results_opt_rep1"
bits0123 = 2

t2_bit3, f2_bit3 = getOutliersCube(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt\\03_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter7.p', '..\\result

In [28]:
train_1 = "results_opt"
train_2 = "results_opt_rep1"
test = "results_opt_rep2"
bits0123 = 2

t3_bit3, f3_bit3 = getOutliersCube(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt\\03_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter7.p', '..\\result

In [31]:
train_1 = "results_opt_rep1"
train_2 = "results_opt_rep2"
test = "results_opt"
bits0123 = 2

e_t1_bit3, e_f1_bit3 = getOutliersEllipsoid(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep2\

err 0.0037996515018941224
err 0.003790748971257665
err 0.0038555290047633736
err 0.0038967647760001054
err 0.003903131022801381
err 0.00407747325569849
err 0.003525182749094603
err 0.003327858563888319
err 0.0033069647457867647
err 0.0030742137011189317
err 0.0032045641498530753
err 0.003295140414503892
err 0.003250360116576335
err 0.0034635509050700156
err 0.0034144727115840716
err 0.003547940182285604
err 0.0037188269897607843
err 0.0038449539705225546
err 0.003325362892291747
err 0.0033997666891709315
err 0.0030190856619426386
err 0.003037477599872529
err 0.003349178490520646
err 0.003223900859379968
err 0.0032211362660929874
err 0.003179033470603429
err 0.0032765275238983912
err 0.003185709290803375
err 0.0029674498319869837
err 0.0029040287882416316
err 0.003080077192456278
err 0.0030590269781572673
err 0.0029265217758443645
err 0.0030336138862347333
err 0.002874274505781004
err 0.002861414533299356
err 0.0028998628919298594
err 0.0028611291952835177
err 0.003000597847506529
err 0

err 0.00147982957059227
err 0.0015416586797366188
err 0.0013904014813008886
err 0.0015442217309874246
err 0.001551308835533792
err 0.0015167167946569845
err 0.001465550233955475
err 0.0014463666408987329
err 0.0014632375945987073
err 0.0016037065731663968
err 0.0017311114078638217
err 0.0016895235118324515
err 0.0016486651121872169
err 0.001580484884505317
err 0.0016430201331853937
err 0.0017838353098202687
err 0.0017178672395582871
err 0.0018140282861416354
err 0.0017364063061555159
err 0.0015494340813808928
err 0.0015196650225139123
err 0.0015168234986643614
err 0.0014325488777698916
err 0.0014682622664263984
err 0.0014687199886965576
err 0.0015549438740317292
err 0.0015003372751892531
err 0.0014867596850534777
err 0.0015493041051971286
err 0.0015436975107620684
err 0.0015106605796681172
err 0.0014965540762582087
err 0.0014152532966976974
err 0.0014868444212478064
err 0.00144850645399545
err 0.001535582644515023
err 0.001531347567519359
err 0.0015163548692166014
err 0.001639998644506

In [33]:
train_1 = "results_opt"
train_2 = "results_opt_rep2"
test = "results_opt_rep1"
bits0123 = 2

e_t2_bit3, e_f2_bit3 = getOutliersEllipsoid(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt\\03_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep2\\03_bioproc_Region0ViableSet_Iter7.p', '..\\result

err 0.0036149241710688937
err 0.0038014276358520715
err 0.003452685994455885
err 0.003473790424759239
err 0.0035555716873512714
err 0.0034284370283807915
err 0.0034685957431326497
err 0.0036381915387981613
err 0.0037490362797092105
err 0.0033190218957679916
err 0.003285470848068812
err 0.003515255207030836
err 0.0034321238977894778
err 0.003228467394065853
err 0.003213695396304825
err 0.003304164334679555
err 0.003353187293998299
err 0.0032464860650488203
err 0.0031033852215733436
err 0.00331368622540557
err 0.0036105633211874206
err 0.0036046216408868073
err 0.0032901255178204103
err 0.003098590349931721
err 0.0031787434690263767
err 0.003468337656700597
err 0.003242049126344003
err 0.0032171409918714594
err 0.0033428376453284933
err 0.0033463563397378476
err 0.00310493074514615
err 0.0034937547840848326
err 0.003375314336370058
err 0.0032872119315717468
err 0.0034559443433725395
err 0.0034312888712955025
err 0.003279460457646378
err 0.0033359045947076236
err 0.0030340097174294117
err

err 0.001738451766522838
err 0.0017971639961363256
err 0.0016928806217350134
err 0.0017769573665116318
err 0.0016515506554095432
err 0.0016912793179129908
err 0.0017107710853090906
err 0.0016220640067855408
err 0.001610596451248424
err 0.0016624818237334824
err 0.001657231625575024
err 0.0016495923063644849
err 0.0017094955200903966
err 0.0016898413491907107
err 0.0015256812362706186
err 0.0016397510063654646
err 0.0016793476729483135
err 0.0017375657793154105
err 0.0016405661687710932
err 0.0016425576034370549
err 0.0015206862262849305
err 0.0014643539805143117
err 0.0015125975727179964
err 0.0014579160080832587
err 0.001513900759138885
err 0.0013533796640012813
err 0.0013515323428448657
err 0.001390645175281625
err 0.0014357745909796995
err 0.0015743118011086188
err 0.0014830639855173482
err 0.0015183447829674138
err 0.0015734594082301354
err 0.0016277620689535506
err 0.0016544516245177905
err 0.001588827211020371
err 0.0015750384412202398
err 0.00162699718437283
err 0.00156788462654

In [36]:
train_1 = "results_opt"
train_2 = "results_opt_rep1"
test = "results_opt_rep2"
bits0123 = 2

e_t3_bit3, e_f3_bit3 = getOutliersEllipsoid(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt\\03_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\03_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\03_bioproc_Region0ViableSet_Iter7.p', '..\\result

err 0.0037344563212162248
err 0.0034418304834427475
err 0.0036063314502823057
err 0.0031686592143815914
err 0.0033867424882580437
err 0.003533583494808954
err 0.00353922296630578
err 0.0034542556362113785
err 0.0032380677092622018
err 0.003508845691766581
err 0.0036043536032440266
err 0.003355456954851723
err 0.0032063807324394285
err 0.003267029594828
err 0.0033065230808484174
err 0.0033284884270575786
err 0.0032009707293725325
err 0.003197309348067338
err 0.003267453140445675
err 0.0032814984161775233
err 0.003223943572201522
err 0.003327298037837924
err 0.003272847309664715
err 0.0032700784895694983
err 0.003128641367381167
err 0.0033447846599898553
err 0.003010008550092962
err 0.00303054191926664
err 0.0031453057199243347
err 0.0031501609301529407
err 0.0033899245279867666
err 0.0033103028756310987
err 0.003176546682894871
err 0.0032846875492076725
err 0.00329389687933149
err 0.0034365512467316767
err 0.002910692569676515
err 0.002917083265395311
err 0.002952247729333861
err 0.0029

err 0.0015765600610660126
err 0.0016476916967344173
err 0.0017636358446841132
err 0.0018308139146537986
err 0.0015779413536068808
err 0.0016655419763766317
err 0.0015622336860157504
err 0.0015546842347519377
err 0.0015525655095302792
err 0.0016298695241075093
err 0.0014774802567976105
err 0.0015522704407209351
err 0.0015606915341954332
err 0.0015748438525597605
err 0.0015952832079402421
err 0.0016476139692454407
err 0.0015047335942450018
err 0.0015473130712397286
err 0.0015872531068262783
err 0.0015815135659047418
err 0.001587400365248734
err 0.0015005604763033717
err 0.001573738111217343
err 0.0015855519993814805
err 0.0016081135918085978
err 0.0016027496265541385
err 0.0016620593416782434
err 0.0016841241327438172
err 0.0016488423997137293
err 0.0015383406631893357
err 0.0015137559339930655
err 0.0014999300175336345
err 0.0015017283886937277
err 0.001548172088801545
err 0.0016170970635322331
err 0.0016133225244663347
err 0.0015767384248164796
err 0.001456899365717893
err 0.0014701971

In [50]:
cube_score_bit3 = ( t1_bit3 + t2_bit3 + t3_bit3)/( (f1_bit3 + f2_bit3 + f3_bit3) + ( t1_bit3 + t2_bit3 + t3_bit3 ))
print("cube_score_bit3:     ", cube_score_bit3)
ellipsoid_score_bit3 = ( e_t1_bit3 + e_t2_bit3 + e_t3_bit3)/((e_f1_bit3+e_f2_bit3+e_f3_bit3)+(e_t1_bit3+e_t2_bit3+e_t3_bit3))
print("ellipsoid_score_bit3:", ellipsoid_score_bit3)

cube_score_bit3:      0.9893035417161873
ellipsoid_score_bit3: 0.9789240155296728


## 4-bit processor: inliers

In [38]:
train_1 = "results_opt_rep1"
train_2 = "results_opt_rep2"
test = "results_opt"
bits0123 = 3

t1_bit4, f1_bit4 = getOutliersCube(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep2\

In [39]:
train_1 = "results_opt"
train_2 = "results_opt_rep2"
test = "results_opt_rep1"
bits0123 = 3

t2_bit4, f2_bit4 = getOutliersCube(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt\\04_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter7.p', '..\\result

In [40]:
train_1 = "results_opt"
train_2 = "results_opt_rep1"
test = "results_opt_rep2"
bits0123 = 3

t3_bit4, f3_bit4 = getOutliersCube(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt\\04_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter7.p', '..\\result

In [41]:
train_1 = "results_opt_rep1"
train_2 = "results_opt_rep2"
test = "results_opt"
bits0123 = 3

e_t1_bit4, e_f1_bit4 = getOutliersEllipsoid(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep2\

err 0.0033688849781401626
err 0.0036514991640192018
err 0.003558572087942568
err 0.0034338917646984197
err 0.003509814455503162
err 0.003754437042069826
err 0.003541265717017633
err 0.0036363562208599903
err 0.0036264549160734964
err 0.0035124525641204176
err 0.003620254125458958
err 0.0034202383891448677
err 0.0035296938333306354
err 0.0033803530253740752
err 0.0032826548981990324
err 0.0035247475615007655
err 0.0036928880098859736
err 0.0035926995979716696
err 0.003544162169984348
err 0.003628538911574633
err 0.003589946911521825
err 0.0035520562297541003
err 0.003837491835558989
err 0.0029270615100539356
err 0.0028762706686030546
err 0.0027547422671797186
err 0.002889493198870493
err 0.003045767232234931
err 0.0030732450093097167
err 0.00333156758891973
err 0.0030963469692419096
err 0.0031406761448775408
err 0.0030636810686640427
err 0.0033218239035024814
err 0.0030853348667858457
err 0.0031222022949956223
err 0.003265909383849075
err 0.0032506315926062223
err 0.0031623451826362707


err 0.001668196781276371
err 0.001714893523437191
err 0.00175175236955096
err 0.0016282124301022134
err 0.0016420985243503134
err 0.0015678916784399837
err 0.0015782916470115148
err 0.0016606418086458831
err 0.0017837954478842621
err 0.0016712968958003916
err 0.001488677178710757
err 0.0014882093589684022
err 0.0013899230949887503
err 0.0014728215138225902
err 0.0013755151632819354
err 0.001433494783106168
err 0.0015126341825337794
err 0.0015143564235270337
err 0.001609857220025051
err 0.0015186753020118245
err 0.0015364096137547194
err 0.0014708981252455132
err 0.0013939337358908764
err 0.00139449408033171
err 0.0015124141360766657
err 0.001488446050544055
err 0.0015825480227328122
err 0.0016360740206193295
err 0.0016097692890370819
err 0.0015984241964364782
err 0.001693742856972166
err 0.0017434539978342606
err 0.0017214879590815872
err 0.001683102191649322
err 0.0017646544580216656
err 0.0016380433147422074
err 0.0016140757093103728
err 0.0015680953985937772
err 0.001423364993503394

In [42]:
train_1 = "results_opt"
train_2 = "results_opt_rep2"
test = "results_opt_rep1"
bits0123 = 3

e_t2_bit4, e_f2_bit4 = getOutliersEllipsoid(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt\\04_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep2\\04_bioproc_Region0ViableSet_Iter7.p', '..\\result

err 0.0035902031652286313
err 0.003735688379938609
err 0.0038998353087192883
err 0.0038143201308722067
err 0.003490674105978425
err 0.003671942315939485
err 0.003608867850241441
err 0.0035215372283999165
err 0.003423117325920203
err 0.003442944913680755
err 0.0034123939656277536
err 0.0035915813403347915
err 0.0036053470155698996
err 0.0028329593670440583
err 0.0027560355700568502
err 0.0029390863948832004
err 0.0030387375401853722
err 0.0032098793595846005
err 0.0030240814034874115
err 0.0031682086718305747
err 0.0030516753427180386
err 0.0033588240770911627
err 0.0033853094171164325
err 0.0032855724737407077
err 0.003351638238610558
err 0.003072968902348917
err 0.003096898578007803
err 0.003162314165766613
err 0.003315131237346379
err 0.003075492511764379
err 0.0030801082536341226
err 0.003218814505591507
err 0.0033861171958694245
err 0.0030281617828928024
err 0.003251904194022609
err 0.003204709506792716
err 0.0029112040747622933
err 0.003114102052802387
err 0.0032159401987867688
er

err 0.001750505865670834
err 0.001754215335666646
err 0.0017502200041690957
err 0.0017060241057697918
err 0.001620745373431566
err 0.0015438325008706648
err 0.0015845979525487795
err 0.001638875948261496
err 0.0016953614199876229
err 0.0017132703247006791
err 0.0015394321081255776
err 0.001543869304062121
err 0.0015720730157685573
err 0.0016185451650730434
err 0.0016852229413014987
err 0.0015820417373843152
err 0.0015534604864033206
err 0.0016732025754554014
err 0.0016212887861873618
err 0.001465377022296712
err 0.0014520387247744409
err 0.0014670705256646292
err 0.0015028499901928178
err 0.0014885123808279404
err 0.001384597282504418
err 0.0014629212918142783
err 0.0013138439250215414
err 0.0014243756374879642
err 0.0014101563563653986
err 0.0013247430688949147
err 0.0013851814388008692
err 0.0013707852408952265
err 0.001443464564665253
err 0.0014701259301596837
err 0.0015102385363122864
err 0.001611324083119352
err 0.00158908251137666
err 0.0016167561565980989
err 0.00163120076270674

In [43]:
train_1 = "results_opt"
train_2 = "results_opt_rep1"
test = "results_opt_rep2"
bits0123 = 3

e_t3_bit4, e_f3_bit4 = getOutliersEllipsoid(train_1 = train_1, train_2 = train_2, test=test, bits0123 = bits0123 )

['..\\results_opt\\04_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter7.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter8.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter9.p', '..\\results_opt\\04_bioproc_Region0ViableSet_Iter10.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter1.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter2.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter3.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter4.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter5.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter6.p', '..\\results_opt_rep1\\04_bioproc_Region0ViableSet_Iter7.p', '..\\result

err 0.0038067656254585453
err 0.003976408568247875
err 0.0038787990455582916
err 0.0035874541324140627
err 0.0036572344427299214
err 0.0031366841436227424
err 0.0032650548949483544
err 0.0032136749439942713
err 0.003455610323784065
err 0.003311205971433378
err 0.0030738988287397433
err 0.003233360676287273
err 0.0035484771493880314
err 0.003043675823295473
err 0.0032289977477354018
err 0.003396478006332684
err 0.00321682063234885
err 0.003256283906041847
err 0.0035126407319664545
err 0.00335296513838421
err 0.0035474063530776434
err 0.003410438209640204
err 0.0035821484973087266
err 0.003746613411990906
err 0.003721439731075211
err 0.0035068711835698463
err 0.0030937623810048808
err 0.0030342332573689
err 0.0031641322702578387
err 0.00312618145664055
err 0.0031584567042548172
err 0.0033065355151550664
err 0.0032804629105560416
err 0.003116398571134482
err 0.0032178684270483524
err 0.0031734035916928098
err 0.003037722511164171
err 0.0032489712594376487
err 0.0034826775477360744
err 0.0

err 0.0015716288448386104
err 0.0016169988739867243
err 0.0016659708006521007
err 0.001686697150541919
err 0.0017728472442723785
err 0.0018326904884283
err 0.001861017603443742
err 0.0019140513419245372
err 0.0020311347289140476
err 0.0016049807074295062
err 0.0015434441750695658
err 0.0016907096278518136
err 0.0015577827504206752
err 0.001644894771495494
err 0.0014424742790572075
err 0.0015355708384555162
err 0.0016138379566712855
err 0.0014932132994454669
err 0.0015422271280552847
err 0.0015460915707792347
err 0.0016246660095937963
err 0.001478229092585353
err 0.0014870959492690947
err 0.0015286316612294073
err 0.0015210350857944636
err 0.0015645257920606568
err 0.0014966359201026513
err 0.0015167004828895568
err 0.0016075524857521257
err 0.001442898922711524
err 0.0015565687885099881
err 0.001509096085094311
err 0.0015198788674728543
err 0.0015533301306026699
err 0.0015564651354499019
err 0.0016025795633612297
err 0.0016117647317859766
err 0.0015225205890544073
err 0.001505549331013

In [47]:
cube_score_bit4 = ( t1_bit4 + t2_bit4 + t3_bit4)/( (f1_bit4 + f2_bit4 + f3_bit4) + ( t1_bit4 + t2_bit4 + t3_bit4 ))
print("cube_score_bit4     :", cube_score_bit4)
ellipsoid_score_bit4 = ( e_t1_bit4 + e_t2_bit4 + e_t3_bit4)/((e_f1_bit4+e_f2_bit4+e_f3_bit4)+(e_t1_bit4+e_t2_bit4+e_t3_bit4))
print("ellipsoid_score_bit4:", ellipsoid_score_bit4)

cube_score_bit4     : 0.9910539119516597
ellipsoid_score_bit4: 0.9849329043396374


In [52]:
# ALL together now:
cube_score_bit1 = ( t1 + t2 + t3)/( (f1 + f2 + f3) + ( t1 + t2 + t3) )
print("cube_score_bit1     :", cube_score_bit1)
ellipsoid_score_bit1 = ( e_t1 + e_t2 + e_t3)/( (e_f1 + e_f2 + e_f3) + ( e_t1 + e_t2 + e_t3) )
print("ellipsoid_score_bit1:", ellipsoid_score_bit1)
print()
cube_score_bit2 = ( t1_bit2 + t2_bit2 + t3_bit2)/( (f1_bit2 + f2_bit2 + f3_bit2) + ( t1_bit2 + t2_bit2 + t3_bit2) )
print("cube_score_bit2     :", cube_score_bit2 )
ellipsoid_score_bit2 = ( e_t1_bit2 + e_t2_bit2 + e_t3_bit2)/((e_f1_bit2+e_f2_bit2+e_f3_bit2)+(e_t1_bit2+e_t2_bit2+e_t3_bit2))
print("ellipsoid_score_bit2:", ellipsoid_score_bit2 )
print()
cube_score_bit3 = ( t1_bit3 + t2_bit3 + t3_bit3)/( (f1_bit3 + f2_bit3 + f3_bit3) + ( t1_bit3 + t2_bit3 + t3_bit3 ))
print("cube_score_bit3:     ", cube_score_bit3)
ellipsoid_score_bit3 = ( e_t1_bit3 + e_t2_bit3 + e_t3_bit3)/((e_f1_bit3+e_f2_bit3+e_f3_bit3)+(e_t1_bit3+e_t2_bit3+e_t3_bit3))
print("ellipsoid_score_bit3:", ellipsoid_score_bit3)
print()
cube_score_bit4 = ( t1_bit4 + t2_bit4 + t3_bit4)/( (f1_bit4 + f2_bit4 + f3_bit4) + ( t1_bit4 + t2_bit4 + t3_bit4 ))
print("cube_score_bit4     :", cube_score_bit4)
ellipsoid_score_bit4 = ( e_t1_bit4 + e_t2_bit4 + e_t3_bit4)/((e_f1_bit4+e_f2_bit4+e_f3_bit4)+(e_t1_bit4+e_t2_bit4+e_t3_bit4))
print("ellipsoid_score_bit4:", ellipsoid_score_bit4)

cube_score_bit1     : 0.97949258328605
ellipsoid_score_bit1: 0.9613358190808138

cube_score_bit2     : 0.9937022645048907
ellipsoid_score_bit2: 0.9858635937290634

cube_score_bit3:      0.9893035417161873
ellipsoid_score_bit3: 0.9789240155296728

cube_score_bit4     : 0.9910539119516597
ellipsoid_score_bit4: 0.9849329043396374
